École d'été GRAPHADON
=======
Travaux pratiques avec pyGSP
-----------

Nous allons voir une introduction pratique au traitement de signaux sur graphes (GSP - Graph Signal Processing). Nous utiliserons le package python [pyGSP](https://github.com/epfl-lts2/pygsp). Ce package, conçu par l'EPFL, permet d'effectuer une grande variété d'opérations pour manipuler des signaux sur graphes, en particulier pour le calcul et la manipulation de la Graph Fourier Transform (GFT) à des fins de filtrage par exemple. 

Vous devrez avoir installé au préalable ce package avec votre gestionnaire de package python préféré: `pip`ou `conda`. Avec `pip`faites la commande `python3 -m pip install pygsp`.

La documentation des fonctionnalités de pyGSP est acessible [ici](https://pygsp.readthedocs.io/en/v0.5.1/reference/index.html).

Commençons par importer les packages dont nous aurons besoin (si certains sont manquants, installez-les également).

In [ ]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import pygsp


$
\renewcommand{\vec}[1]{\boldsymbol{\mathsf{#1}}}
\newcommand{\G}{\mathcal{G}}
\newcommand{\V}{\mathcal{V}}
\newcommand{\E}{\mathcal{E}}
\newcommand{\x}{\mathbf{x}}
$

## 1. Création de graphes
Dans pyGSP, un graphe est encodé sous la forme de sa matrice d'adjacence pondérée. Un graphe $G=(\V,\E)$, constitué d'un ensemble de noeuds $\V$ et d'arêtes $\E$, sera donc représenté par une matrice d'adjacence pondérée $\vec{W}\in\mathbb{R}^{N\times N}$ avec $N=|\V|$.

Le module [graphs](https://pygsp.readthedocs.io/en/stable/reference/graphs.html) de pyGSP permet de créer de nombreux graphes. Ces graphes peuvent avoir un plongement naturel dans un domaine Euclidien 2D ou 3D, ou ne pas en avoir. Il est également possible de générer des graphes aléatoirement, d'en créer par des graphes de proximité sur des nuages de point et voir même directement en manipulant la matrice $\vec{W}$. Nous allons explorer ces différentes possibilités.

Lorsqu'un graphe a été créé et stocké dans une variable `G` , on peut accéder à ses nombres de noeuds et d'arêtes avec `G.N`et `G.Ne`.
Commençons par créer un graphe à partir de sa matrice d'adjacence pondérée.

Dans pyGSP, $\vec{W}$ est stockée sous la forme d'une matrice sparse (seuls les coefficients non nuls sont stockés). Pour cela nous allons créer aléatoirement des arêtes entre les noeuds du graphes. On peut ensuite afficher les propriétés du graphes (connexe ? dirigé ?). Dans la cellule suivante, le graphe créé est dirigé. Trouvez comment modifier $\vec{W}$ pour le rendre non-dirigé puis re-créez le graphe. On peut aussi accéder au graphe à partir de se représentation sous forme de liste d'adjacence et non matricielle.

Pour afficher le graphe, on peut afficher sa matrice $\vec{W}$. Comme elle est stockée sous forme d'une matrice sparse, il faut la rendre dense (avec `todense()`)pour l'afficher sous forme d'une matrice avec `plt.matshow`.
Enfin, pyGSP peut aussi directement afficher un graphe avec l'instruction `G.plot()`. Cela requiert cependant que les noeuds du graphe aient des coordonnées Euclidiennes 2D ou 3D. Si ce n'est pas le cas, on peut leur en donner des fictives avec `G.set_coordinates()`.

In [ ]:
W = sp.sparse.random(100, 100, 0.02)  # Sparse graph.
W.setdiag(0)
W.setdiag(0) # on met la diagonale à 0, pas de self-loop

G = pygsp.graphs.Graph(W)

print(G.W)
print('{} nodes, {} edges'.format(G.N, G.Ne))
print('Connected: {}'.format(G.is_connected()))
print('Directed: {}'.format(G.is_directed()))

#TODO : rendez la matrice W symmétrique, re-créez le graphe, et vérifiez qu'il n'est plus dirigé


#récupérer la liste d'adjacence
v_in, v_out, weights = G.get_edge_list()
print(v_in.shape, v_out.shape, weights.shape)

#On affiche la matrice W
plt.spy(G.W, markersize=1)
plt.figure()
#On affiche la matrice W sous forme de matrice
plt.matshow(G.W.todense())
plt.figure()

G.set_coordinates()  # on donne des coordonnées aux noeuds
G.plot()



Il y a des graphes pré-existants dans pyGSP. Créez (dans des variables nommées `G[1-4]`) des graphes Minnesota, Sensor (64 noeuds), StochasticBlockModel (reprenez l'exemple de la documentation, et donnez des coordonnées aux noeuds comme indiqué), et Grid2d (de taille $10\times 10$). Pour chacun, affichez la matrice $\vec{W}$ et le graphe. Ecrivez une fonction qui permet de le faire et appellez-là pour chaque graphe créé.

On peut également construire un graphe à partir d'un ensemble de points de $\mathbb{R}^d$. 

Soit $\vec{X}$ une matrice contenant les données $\vec{X} = [\vec{x}_1, \ldots, \vec{x}_N]^\intercal \in \mathbb{R}^{N \times d}$, où chaque $\vec{x} \in \mathbb{R}^d$ est une instance de la base de données et est représentée par un vecteur de taille $d$. 

Commençez par créer de tels points (100) dans un espace de dimension 2 avec `np.random.uniform` et affichez les avec `plt.scatter`.

On peut ensuite créer un graphe de proximité qui connecte les points par des arêtes si ils sont proches selon un critère donné.
Dans pyGSP, on peut utiliser le graphe des k-plus proches voisins (kppv, chaque point est relié à ses $k$ plus proches voisins) ou l'epsilon graphe (chaque point est relié aux points contenus dans une boule de rayon $\epsilon$ autour de lui). Les arêtes sont ensuite généralemment pondérées par un noyau positif donnant la similarité entre deux points (proche de 0 si différents, proche de 1 si similaires):
$$\vec{W}[i,j] = k(\vec{x}_i, \vec{x}_j)$$
et qui est souvent le noyau Gaussien:
$$k(\vec{x}_i, \vec{x}_j) = \exp \left(-\frac{d^2(\vec{x}_i, \vec{x}_j)}{\sigma^2} \right)$$
Créez un 2-ppv sur les points précédemment créés et pondérez les arêtes avec un noyau Gaussien de $\sigma=0.1$.
Affichez $\vec{W}$ et le graphe. Affichez ensuite un histogramme (`plt.hist`) des poids (`G.W.data`) et un histogramme des degrés (`G.d`, en termes d'arêtes incidentes). Les histogrammes auront $10$ bins. Qu'en concluez-vous ?

## 2. Signaux sur graphes

Un signal sur graphe est une fonction $f: \mathcal{V} \rightarrow \mathbb{R}$ qui associe une valeur scalaire à chaque noeud $v\in\V$ d'un graphe $G$. L'ensemble des valeurs peut être vue comme un vecteur $\vec{x}_i=f(v_i) \in \R^N$, avec $N = |\V|$ le nombre de noeuds.

Commençez par créer un signal aléatoire pour le graphe de Community `G3` précédemment créé (avec `np.random.normal`), puis affichez le (il faut utiliser la fonction `graph.plot_signal`):

### 2.1 Gradient, divergence, et Laplacien combinatoire

Le gradient $\vec{(\nabla_wf)}$ d'un signal $f$ sur un graphe $G$ est un signal sur les arêtes défini par 

$$\vec{(\nabla_wf)(v_i)} = [w(v_i,v_j)^{1/2}(f(v_j)-f(v_i))\,:\, v_j\in\mathcal{V}]^\intercal.$$

en PyGSP on peut l'obtenir facilement en appelant la commande `graph.compute_differential_operator`. Il est alors disponible via `graph.D`.


Calculez le gradient sur le signal aléatoire précédent et vérifiez que la cardinalité du gradient est bien égale aux nombre d'arêtes du graphe.

De la même manière on peut définir l'opérateur de divergence pour un signal $H$ défini sur les arêtes et cela renvoie un signal défini sur les noeuds:
$$div_w(H)(v_i)=\sum_{v_j\sim v_i}{w(v_i,v_j)^{1/2}(H(v_i,v_j)-H(v_j,v_i))}$$

Calculez la divergence du gradient, vérifiez également que la cardinalité de la divergence est égale au nombre de noeuds du graphe.

En PyGSP, le gradient et la divergence peuvent être calculés directement avec `G.grad` et `G.div` sur un signal. Vérifiez-le.


Par définition, le Laplacien (aussi appellé 2-Laplacien ou Laplacien combinatoire), est la divergence du gradient. En pyGSP, le Laplacien d'un graphe `G` s'obtient par `G.L`. 

Vérifiez que l'on obtient bien le même signal avec `G.L` qu'en calculant le Laplacien de notre signal par la divergence du gradient (utilisez `np.allclose` pour les comparer). 

Plottez les deux signaux calculés sur le graphe.

### 2.2 Le Laplacien normalisé
Nous venons de voir que le Laplacien combinatoire $\vec{L}=\vec{D}-\vec{W}$ peut être obtenu à partir de la divergence du gradient. Notez que ces définitions ne sont valables que pour le Laplacien combinatoire. Pour le Laplacien normalisé, il s'obtient en normalisant le Laplacien par la matrice des degrés. On obtient alors $\mathcal{L}=\vec{D}^{-1/2} \cdot \vec{L} \cdot \vec{D}^{-1/2}=\vec{D}^{-1/2} \cdot (\vec{D}-\vec{W}) \cdot \vec{D}^{-1/2}=\vec{I}-\vec{D}^{-1/2} \cdot \vec{W} \cdot \vec{D}^{-1/2}$.

Le gradient correspondant est 
$$\vec{(\nabla_wf)(v_i)} = \left[w(v_i,v_j)^{1/2}\left(\frac{f(v_j)}{\sqrt{d(v_j)}}-\frac{f(v_i)}{\sqrt{d(v_i)}}\right)\,:\, v_j\in\V\right]^\intercal.$$
avec $d(v_i)=\sum\limits_{v_j\sim v_i}w_{ij}$ le degré du noeud $v_i$.
En PyGSP on peut l'obtenir facilement en apellant la commande `G.compute_laplacian('normalized')`. Le Laplacien combinatoire s'obtenant avec `G.compute_laplacian('combinatorial')`. Une fois calculé, le Laplacien (combinatoire ou normalisé) s'obtient avec `G.L`. 

Calculez ces deux Laplaciens pour le signal précédent et affichez les. Quelles différences constatez-vous ?

### 2.3 Aspect lisse d'un signal
On peut caractériser l'aspect lisse d'un signal sur graphe par son Graph Laplacian Regularizer (GLR):
$$\vec{f}^\intercal\vec{L}\vec{f}=\tfrac{1}{2}\sum_{v_i\in \V}\|\vec{(\nabla_wf)(v_i)}\|^2_2=\tfrac{1}{2}\sum\limits_{v_i\in \mathcal{V}}\left[\sum\limits_{v_j \sim v_i}w_{ij}(f(v_j)-f(v_i))^2\right]$$

Calculez cette quantité pour le signal précédent avec un Laplacien combinatoire. Ce signal est-il lisse ?

Quel est le signal sur graphe $\vec{f}$ qui est le plus lisse c-à-d celui pour lequel $\vec{f}^\intercal\vec{L}\vec{f}=0$ avec le Laplacien combinatoire ? 

Est-ce aussi le cas avec le Laplacien normalisé ? 

Si non quel signal faut-il considérer ?

Vérifiez cela en faisant les calculs.

Comme précédemment nous avons généré un graphe de $3$ communautés, nous pouvons aussi considérer un signal comme fonction d'appartenance à ceux-ci:
$$ x[i] =
\begin{cases}
    -1 &\text{si } i \in S_1, \\
    0  &\text{si } i \in S_2, \\
    1  &\text{si } i \in S_3
\end{cases}
$$
avec $S_i$ les noeuds de la communauté $i$.

Les clusters peuvent être récupérés par `x=np.copy(G3.info['node_com'].astype(float))`


Créez ce signal $\vec{x}$, affichez-le et calculez son GLR. Le signal est-il plus lisse que le signal aléatoire ?


Enfin, si nous souhaitons trouver le signal non-trivial (c-à-d non nul et non unitaire) le plus lisse, il nous faut résoudre:
$$\vec{x}^\star = \argmin_{\vec{x} \in \R^N} \vec{x}^\intercal \vec{L} \vec{x}, \ \text{ s.t. } \ \vec{x} \neq \vec{0} \ \text{ and } \ \vec{x}^\intercal  \vec{1} = 0,$$
avec $\vec{0}$ un vecteur nul composé de $0$, et $\vec{1}$ un vecteur unitaire composé de $1$. La seconde contrainte impose que le signal soit orthogonal au vecteur unitaire. Comme le vecteur unitaire est le premier vecteur propre du Laplacien combinatoire (associé à la valeur propre $0$), la solution au problème de minimisation est le second vecteur propre du Laplacien combinatoire. 

Calculez ce signal avec la méthode `sparse.linalg.eigsh` du package scipy, puis calculez son GLR, pour le $\vec{L}$ et $\mathcal{L}$. Lequel est le plus lisse ?

Calculez ensuite des 3 premiers vecteurs propres et valeurs propres du Laplacien combinatoire (toujours avec ` sp.sparse.linalg.eigsh`).
Affichez pour chaque vecteur propre obtenu : le signal sur le graphe, le signal binarisé selon la fonction indicatrice (0 si négatif, 1 si positif), la valeur propre du signal, la GLR du signal.

Que constatez-vous ?


### 3. La Graph Fourier Transform (GFT)

Comme en traitement du signal classique, la transformée de Fourier sur graphe joue un rôle essentiel en traitement de signaux sur graphes.

La base de Fourier sur graphe est définie par $\vec{U} = [\vec{u}_1, \ldots, \vec{u}_N] \in \R^{N \times N}$, où les colonnes de $\vec{U}$ sont les vecteurs propres du Laplacien $\vec{L}$. Il peut être calculé en pyGSP par la fonction`compute_fourier_basis()`. Attention, cela implique une décomposition en vecteurs propres de $\vec{L}$ qui a une complexité en $\mathcal{O}(N^3)$.

On peut analyser le contenu spectral d'un signal sur graphe via sa GFT. Cela indique comment le signal  varie en fonction de sa connecivité: passe-bas, passe-haut.

Commençons par calculer la GFT pour le graphe $G3$ et affichons les $7$ premiers modes de fourier.


In [ ]:
fig, axes = plt.subplots(1, 7, figsize=(15, 2.5))
G3.compute_fourier_basis()

limits = [f(G3.U[:, :len(axes)]) for f in (np.min, np.max)]

for i, ax in enumerate(axes):
    G3.plot_signal(G3.U[:, i], limits=limits, colorbar=False, vertex_size=50, ax=ax)
    ax.set_title(f'eigenvector $u_{i+1}$')
    ax.set_axis_off()


La GFT (en pyGSP: la fonction `G.gft()`) d'un signal $\vec{f}$ est donnée par 
$$\hat{\vec{x}} = \mathcal{F}\{\vec{x}\} = \vec{U}^\intercal \vec{x} \in \R^N,$$
où $\vec{U}$ est la base de Fourier sur graphe. La réponse à la fréquence $\lambda_i$ est donnée par
$$\hat{\vec{x}}[i] = \langle \vec{u}_i, \vec{x} \rangle.$$

La GFT inverse (en pyGSP: la fonction `G.igft()`) est donnée par 
$$\vec{x} = \mathcal{F}^{-1}\{\hat{\vec{x}}\} = \vec{U} \hat{\vec{x}} \in \R^N.$$


On peut aussi regarder l'aspect lisse d'un signal en affichant leur GFT (l'interprétation se fait alors dans le domaine spectral). 

Faîtes le pour le signal aléatoire et le signal de partition précedemment créés pour le graphe $G2$.

### 4. Filtrage de Signaux sur graphes dans le domaine spectral

Un signal sur graphe $\vec{x}$ est filtré par l'opération 
$$\vec{y} = \vec{U} \hat{g}(\vec{\Lambda}) \vec{U}^\intercal \, \vec{x} = \hat{g}(\vec{U} \vec{\Lambda} \vec{U}^\intercal) \, \vec{x} = \hat{g}(\vec{L}) \, \vec{x},$$

avec $\hat{g}(\cdot)$ qui est le noyau du filtre défini dans le domaine spectral comme une fonction des valeurs propres. 
Le filtrage peut alors se réaliser selon les étapes suivantes :

1. On obtient la représentation specrale du signal via la GFT: 
$\hat{\vec{x}} = \vec{U}^\intercal \, \vec{x}$.
2. On filtre le signal dans le domaine spectral en multipliant chaque coefficient de Fourier avec le coefficient correspondant du filtre, c-à-d $\hat{\vec{y}} = \hat{g}(\vec{\Lambda})\hat{\vec{x}} $
3. On retourne dans le domaine nodal en prenant la GFT inverse du signal filtré : $\vec{y} = \vec{U}\hat{\vec{y}}$




### 4.1 Premier filtrage : l'équation de la chaleur
Commençons par un premier filtre qui réalise une opération simple de filtrage en utilisant le noyau de la chaleur $h(\lambda)$ défini par:
$$h_\tau(\lambda)=\exp^{-\tau\lambda}.$$
La cellule suivante en donne une illustration. On place un signal à 1 sur le noeud numéro 10 et on veut apppliquer l'équation de la chaleur. 

Ce filtre est disponible par `pygsp.filters.Heat`. Appliquez-le pour des valeurs de $\tau$ parmi $0, 5, 100$ et affichez les signaux filtrés obtenus et la GLR pour chacun.

Que constatez-vous ?

In [ ]:
G5 = pygsp.graphs.Sensor(N=256,seed=42)
G5.compute_fourier_basis()

s = np.zeros(G5.N)
DELTA = 20
s[DELTA] = 1


    
        


### 4.2 Régulariation de Tikhonov 

Définissons un filtre passe-bas par 
$$ g(\lambda) = \frac{1}{1+\tau\lambda} $$

Étant donné $\vec{x}_\text{noisy}$ une version bruitée d'un signal , on peut le débruiter en appliquant le filtre passe-bas $g$:
$$ \vec{x}_\text{denoised} = \vec{U}g(\vec{\Lambda})\vec{U}^\top \vec{x}_{\text{noisy}}, $$
ce qui correspond à l'opération de filtrage suivante : $g(L)  = (I + \tau L)^{-1}$ (la solution de la GLR avec attache aux données en forme analytique).

On peut définir un filtre `g`quelconque à partir d'une fonction python et le définir comme un filtre pygsp avec `gf=pygs.filters.Filter(...)`. Ce filtre peut ensuite s'appliquer avec `gf.filter` sur un signal. Apppliquez ce filtre passe-bas sur le signal bruité. Affichez les signaux originaux, bruités et débruités et caculez la MSE entre signal orignal et bruité, ainsi qu'entre signal original et débruité. Qu'en concluez-vous ?

In [ ]:

tau = 1
def g(x):
    return 1. / (1. + tau * x)
gf = pygsp.filters.Filter(G5, g)

#définissons un signal sur le graphe
x=np.copysign(np.ones(G5.N), G5.U[:, 3])
#ajoutons du bruit
x_noisy = x +  np.random.randn(G5.N)
#filtrons le signal bruité



### 5. Filtrage de Signaux sur graphes dans le domaine nodal


Un signal peut être filtré dans le domaine nodal en appliquant des moyennes itératives, cela supprime le bruit mais a tendance à lisser fortement le signal. 

On peut utiliser la matrice des poids $\vec{W}$ normalisés par la matrice des degrés $\vec{D}$ pour filtrer notre signal $\vec{x}$ par : $$\vec{x}^{i+1}=\vec{x}^{i}\left(\vec{D}^{-1}\vec{W}\right)^\intercal$$

Modifiez le code ci-dessous afin de faire plusieurs itérations (10) de cette opération sur le signal bruité précédent, de calculer la MSE entre le signal original et sa version lissée. Faites un plot des valeurs de la MSE selon les itérations et affichez le signal débruité obtenu à la fin des itérations.


In [ ]:
#on calcule le vecteur les degrés des noeuds
d = np.ravel((G5.W.sum(0)))
#On créee la matrice diagonale D
n=G5.N
#D = sp.sparse.spdiags(d, 0, n, n)
#On calcule l'inverse de D
iD = sp.sparse.coo_matrix((1/d, (np.arange(0,n),np.arange(0,n))))
#On calcule D^{-1}W
tW = iD.dot(G5.W)



Il est également possible de filtrer un signal dans le domaine nodal en exploitant une régularisation composée d'un terme d'attache aux données et d'un terme de régularité.
L'attache aux données est généralement mesurée par une différence en norme $L_2$ entre le signal régularisé et le signal original bruité. Le terme de régularité est 
multiplié par une constante de régularisation $\gamma$ qui permet de préciser le niveau de régularité à imposer (c-à-d si l'on souhaite rester proche ou non du signal original (bruité)).
Le terme de régularisation peut alors être soit
- Une régularisation de type GLR (Graph Laplacian Regularization, souvent dite de "Tikhonov"). Ce terme exploite un régulariseur quadratique (différentiable) reposant sur la norme $L_2$ du gradient sur graphe. Nous l'avons vu précédemment. On va donc chercher à minimiser $$\min\limits_{\vec{f}}\gamma\sum\limits_{v_i\in \V}\|\vec{(\nabla_w f)(v_i)}\|^2_2+\|\vec{f}-\vec{f}^0\|_2^2$$
- Une régularisation de type GTV (Graph Total Variation). Ce terme repose sur la norme $L_1$ du gradient sur graphe et force l'obtention de solutions constantes par morceaux. On va donc chercher à minimiser $$\min\limits_{\vec{f}}\gamma\sum\limits_{v_i\in \V}\|\vec{(\nabla_w f)(v_i)}\|_1+\|\vec{f}-\vec{f}^0\|_2^2$$


Commençons avec la GLR. Elle peut être résolue directement sous forme analytique et la solution est $(I + \gamma L)^{-1}\vec{f}$. Effectuez la résolution (à l'aide de `sp.sparse.linalg.inv`) avec $\gamma=1$ pour régulariser le signal bruité précédent. Affichez-le résultat et calculez la MSE, puis comparez avec celle obtenue dans le domaine spectral. Comme la matrice est très sparse, cela peut se résoudre plus efficacement avec un gradient conjugué (`sp.sparse.linalg.cg`). Vérifiez-le.

Passons maintenant à la GTV regularisation. Contrairement à la GLR qui a une solution sous forme analytique, ce n'est pas le cas de la GTV qui est non différentiable à cause de la norme $L_1$. Nous allons utiliser un solveur primal-dual pour résoudre ce problème d'optimisation. Celui considéré provient du package `pyunlocbox`. Il s'agit du solveur `pyunlocbox.solvers.mlfbf`. C'est une méthode primale duale, ce qui signifie que le terme de régularisation va être écrit selon la variable duale $\vec{u}=\vec{(\nabla_w f)}$. Le code ci-dessous permet de réaliser la mininisation et d'obtenir le résultat. Affichez-le ainsi que la MSE. Qu'en conclure par rapport à la régularisation GLR ?

In [ ]:
import pyunlocbox
#Definition du paramètre gamma
gamma=1
#Definition des fonctions du problème
d = pyunlocbox.functions.dummy()
r = pyunlocbox.functions.norm_l1()
f = pyunlocbox.functions.norm_l2(y=x_noisy,lambda_=gamma)
#Definition du solveur
G5.compute_differential_operator()
L = G5.D.toarray()
step = 1 / (1 + np.linalg.norm(L))
solver = pyunlocbox.solvers.mlfbf(L=L, step=step)
#Definition du problème
x0=x_noisy.copy()
pb = pyunlocbox.solvers.solve([d, r, f], solver=solver, x0=x0, rtol=0, maxit=1000)
x_res = pb['sol']
G5.plot_signal(x_res)
print('MSE orig vs denoised {:.5f}'.format(np.linalg.norm(x - x_res)))
#Vous devriez obtenir une erreur plus faible qu'avec les méthodes précédentes


Nous en avons terminé avec les bases du traitement su signal sur graphes.

### 6. Application au débruitage d'un maillage 3D

Essayons maintenant d'appliquer ce que nous venons de voir sur des données plus réelles. Nous allons considérer un maillage 3D représentant un éléphant (fichier `elephant.off`). Le maillage est au format `.off`et le package `trimesh`permet de le manipuler. Sa documentation est visible [https://trimesh.org/](ici).

Chargez le maillage (`trimesh.load`) et affichez-le (`mesh.show()`).

Ajoutez du bruit aux coordonnées des noeuds du maillage avec `trimesh.permutate.noise` et affichez le maillage bruité obtenu.

À vous maintenant ! récupérez la matrice d'adjacence du maillage (avec `trimesh.graph.edges_to_coo`) puis appliquez une régularisation GLR pour débruiter le maillage. Affichez le résultat. Est-on proche du maillage original ? Vérifiez avec le calcul de la MSE.